In [ ]:
import os
import math
import time
import heapq
import random
import json
import zipfile
import io
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from google.colab import files
import matplotlib.pyplot as plt


In [ ]:
# Upload maze zip
uploaded = files.upload()
for fname in uploaded:
    with zipfile.ZipFile(io.BytesIO(uploaded[fname])) as zip_ref:
        zip_ref.extractall("imperfect_maze")

input_dir = "imperfect_maze/imperfect_maze"
maze_files = [os.path.join(input_dir, f) for f in sorted(os.listdir(input_dir)) if f.endswith(".txt")]


In [ ]:
# Split maze files into training and testing sets
def split_mazes(maze_files, train_frac=0.2, seed=42):
    random.seed(seed)
    random.shuffle(maze_files)
    split = int(len(maze_files) * train_frac)
    return maze_files[:split], maze_files[split:]

train_mazes, test_mazes = split_mazes(maze_files)
print(f"{len(train_mazes)} training mazes, {len(test_mazes)} testing mazes")


In [ ]:
def load_maze(path):
    return np.loadtxt(path, dtype=int)

In [ ]:
def q_heuristic(state):
    return -max(Q.get(state, {}).values(), default=0)

def manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def manhattan_heuristic(state):
    return manhattan(state, goal)

def euclidean_heuristic(state):
    return ((state[0] - goal[0]) ** 2 + (state[1] - goal[1]) ** 2) ** 0.5

In [ ]:
class TabularQLearning:
    def __init__(self, actions, discount=1.0, explorationProb=0.2, initialQ=0.0, step_size=0.5):
        self.Q = {}
        self.actions = actions
        self.discount = discount
        self.explorationProb = explorationProb
        self.initialQ = initialQ
        self.step_size = step_size

    def getQ(self, state, action):
        return self.Q.get(state, {}).get(action, self.initialQ)

    def choose_action(self, state):
        if random.random() < self.explorationProb:
            return random.choice(self.actions)
        qs = [(a, self.getQ(state, a)) for a in self.actions]
        maxQ = max(qs, key=lambda x: x[1])[1]
        best_actions = [a for a, q in qs if q == maxQ]
        return random.choice(best_actions)

    def update(self, state, action, reward, next_state):
        oldQ = self.getQ(state, action)
        max_future_q = max([self.getQ(next_state, a) for a in self.actions], default=0)
        target = reward + self.discount * max_future_q
        newQ = oldQ + self.step_size * (target - oldQ)
        if state not in self.Q:
            self.Q[state] = {}
        self.Q[state][action] = newQ


In [ ]:
def astar(maze, start, goal, heuristic):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    while frontier:
        _, current = heapq.heappop(frontier)
        if current == goal:
            break
        for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            next_state = (current[0] + dr, current[1] + dc)
            if (
                0 <= next_state[0] < maze.shape[0]
                and 0 <= next_state[1] < maze.shape[1]
                and maze[next_state] == 0
            ):
                new_cost = cost_so_far[current] + 1
                if next_state not in cost_so_far or new_cost < cost_so_far[next_state]:
                    cost_so_far[next_state] = new_cost
                    priority = new_cost + heuristic(next_state)
                    heapq.heappush(frontier, (priority, next_state))
                    came_from[next_state] = current
    return came_from, cost_so_far


In [ ]:
def train_q_learning_reversed(maze, goal, actions,
                                      discount, explorationProb, step_size,
                                      num_sample_starts, step_multiplier):
    q = TabularQLearning(actions, discount=discount,
                         explorationProb=explorationProb,
                         step_size=step_size)
    height, width = maze.shape
    max_steps = step_multiplier * (height + width)

    def is_valid(pos):
        r, c = pos
        return 0 <= r < height and 0 <= c < width and maze[r, c] == 0

    free_cells = [tuple(cell) for cell in np.argwhere(maze == 0)]
    sample_starts = random.sample(free_cells, min(num_sample_starts, len(free_cells)))

    episodes = 500
    for ep in range(episodes):
        state = goal
        for _ in range(max_steps):
            action = q.choose_action(state)
            next_state = (state[0] + action[0], state[1] + action[1])
            if not is_valid(next_state):
                continue
            reward = 0 if next_state in sample_starts else -1
            q.update(state, action, reward, next_state)
            state = next_state
    return q.Q


In [ ]:
def evaluate_runtime(config, maze, start, goal, actions):
    Q = train_q_learning_reversed(
        maze, goal, actions,
        discount=config["discount"],
        explorationProb=config["explorationProb"],
        step_size=config["step_size"],
        num_sample_starts=config["num_sample_starts"],
        step_multiplier=config["step_multiplier"]
    )

    def q_heuristic(state):
        return -max(Q.get(state, {}).values(), default=0)

    start_time = time.time()
    came_from, cost = astar(maze, start, goal, q_heuristic)
    return time.time() - start_time


In [ ]:
def run_single_map_search(path, actions, n_trials=30):
    maze = load_maze(path)
    free_cells = list(zip(*np.where(maze == 0)))
    if len(free_cells) < 2:
        return None
    start = free_cells[0]
    goal = free_cells[-1]

    result = []
    for _ in range(n_trials):
        config = {
            "discount": round(random.uniform(0.8, 1.0), 2),
            "explorationProb": round(random.uniform(0.1, 0.7), 2),
            "step_size": round(random.uniform(0.1, 0.9), 2),
            "num_sample_starts": random.randint(3, 20),
            "step_multiplier": random.randint(5, 20)
        }
        try:
            runtime = evaluate_runtime(config, maze, start, goal, actions)
            config["runtime"] = runtime
            config["maze"] = os.path.basename(path)
            result.append(config)
        except Exception as e:
            print(f"Failed on {path}: {e}")
    return result


In [ ]:
def run_hyperparameter_search(train_mazes, n_maps=30, n_trials_per_map=30):
    selected_paths = random.sample(maze_files, min(n_maps, len(maze_files)))
    actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    all_results = []

    for map_idx, path in enumerate(selected_paths):
        print(f"\n Running map {map_idx + 1}/{len(selected_paths)}: {os.path.basename(path)}")
        maze = load_maze(path)
        free_cells = list(zip(*np.where(maze == 0)))
        if len(free_cells) < 2:
            print("Skipping map due to insufficient free space.")
            continue
        start = free_cells[0]
        goal = free_cells[-1]

        for trial in range(n_trials_per_map):
            config = {
                "discount": round(random.uniform(0.8, 1.0), 2),
                "explorationProb": round(random.uniform(0.1, 0.7), 2),
                "step_size": round(random.uniform(0.1, 0.9), 2),
                "num_sample_starts": random.randint(3, 20),
                "step_multiplier": random.randint(5, 20)
            }
            try:
                print(f"Trial {trial + 1}/{n_trials_per_map}", end="\\r")
                runtime = evaluate_runtime(config, maze, start, goal, actions)
                config["runtime"] = runtime
                config["maze"] = os.path.basename(path)
                all_results.append(config)
            except Exception as e:
                print(f"Trial {trial + 1} failed: {e}")

    df = pd.DataFrame(all_results)
    df.to_csv("hyperparameter_results.csv", index=False)
    with open("hyperparameter_results.json", "w") as f:
        json.dump(all_results, f, indent=2)
    return df



In [ ]:
results_df = run_hyperparameter_search(
    train_mazes,
    n_maps=30,
    n_trials_per_map=30
)
results_df.head()
# Training time:

In [ ]:
results = []

# Hardcoded best config
best_config = {
    "discount": 0.92,
    "explorationProb": 0.49,
    "step_size": 0.65,
    "num_sample_starts": 3,
    "step_multiplier": 10
}
actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

for i, path in enumerate(test_mazes):
    maze = load_maze(path)
    free_cells = list(zip(*np.where(maze == 0)))
    if len(free_cells) < 4:
        print("  Skipped (not enough free cells)")
        continue

    goal = free_cells[-1]
    sample_starts = random.sample(free_cells, 3)

    # Train Q-learning once
    Q = train_q_learning_reversed(
        maze, goal, actions,
        discount=best_config["discount"],
        explorationProb=best_config["explorationProb"],
        step_size=best_config["step_size"],
        num_sample_starts=best_config["num_sample_starts"],
        step_multiplier=best_config["step_multiplier"]
    )

    heuristics = {
        "Manhattan": manhattan_heuristic,
        "Euclidean": euclidean_heuristic,
        "Q-learned": q_heuristic
    }

    for name, heuristic in heuristics.items():
        total_runtime = 0
        total_path_length = 0
        total_nodes_expanded = 0

        for start in sample_starts:
            start_time = time.time()
            came_from, cost = astar(maze, start, goal, heuristic)
            runtime = time.time() - start_time
            path_length = cost.get(goal, float('inf'))
            nodes_expanded = len(cost)

            total_runtime += runtime
            total_path_length += path_length
            total_nodes_expanded += nodes_expanded

        results.append({
            "maze": os.path.basename(path),
            "heuristic": name,
            "runtime": total_runtime,
            "path_length": total_path_length,
            "nodes_expanded": total_nodes_expanded
        })

# Save results
df = pd.DataFrame(results)
df.to_csv("final_heuristic_comparison.csv", index=False)
with open("final_heuristic_comparison.json", "w") as f:
    json.dump(results, f, indent=2)

print("Evaluation complete. Results saved.")

In [ ]:
def visualize_heuristics_on_sampled_paths(test_mazes, best_config, num_starts=3):
    actions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    results = []

    path = random.choice(test_mazes)
    maze = load_maze(path)
    free_cells = list(zip(*np.where(maze == 0)))
    if len(free_cells) < num_starts + 1:
        print("Not enough free cells.")
        return

    goal = free_cells[-1]
    sample_starts = random.sample(free_cells, num_starts)

    Q = train_q_learning_reversed(
        maze, goal, actions,
        discount=best_config["discount"],
        explorationProb=best_config["explorationProb"],
        step_size=best_config["step_size"],
        num_sample_starts=best_config["num_sample_starts"],
        step_multiplier=best_config["step_multiplier"]
    )

    heuristics = {
        "Manhattan": manhattan_heuristic,
        "Euclidean": euclidean_heuristic,
        "Q-learned": q_heuristic
    }

    for name, h in heuristics.items():
        total_runtime = 0
        path_colors = ["red", "green", "blue"]
        all_paths = []

        # Compute heatmap values
        values = np.full(maze.shape, np.nan)
        for r in range(maze.shape[0]):
            for c in range(maze.shape[1]):
                if maze[r, c] == 0:
                    try:
                        values[r, c] = h((r, c))
                    except:
                        values[r, c] = np.nan

        for idx, start in enumerate(sample_starts):
            start_time = time.time()
            came_from, cost = astar(maze, start, goal, h)
            runtime = time.time() - start_time
            total_runtime += runtime

            path_points = []
            current = goal
            while current in came_from and came_from[current] is not None:
                path_points.append(current)
                current = came_from[current]
            if current == start:
                path_points.append(start)
                path_points = path_points[::-1]
                all_paths.append((idx + 1, path_points))
            else:
                all_paths.append((idx + 1, []))

        plt.figure(figsize=(8, 6))
        plt.imshow(values, cmap="viridis")
        plt.title(f"{name} Heuristic\nTotal Runtime: {total_runtime:.4f}s")
        plt.colorbar()

        for (label, path_points), color in zip(all_paths, path_colors):
            if path_points:
                y, x = zip(*path_points)
                plt.plot(x, y, color=color, label=f"Path {label}")
                plt.scatter(x[0], y[0], color=color, edgecolors='black', label=f"Start {label}", marker='o')
            else:
                print(f"⚠️ Path {label} not found.")

        plt.scatter(goal[1], goal[0], c="white", marker="X", s=100, label="Goal")
        plt.legend()
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f"heatmap_paths_{name.lower()}.png")
        plt.show()

    print(f"Done for maze: {os.path.basename(path)}")



sampled_paths = random.sample(test_mazes, 5)

for i, path in enumerate(sampled_paths):
    print(f"\n Running map {i+1}/5: {os.path.basename(path)}")
    visualize_heuristics_on_sampled_paths([path], best_config)

In [ ]:
def plot_runtime_progression(results_df):
    sorted_df = results_df.sort_values("runtime").reset_index(drop=True)
    plt.figure()
    plt.plot(sorted_df.index, sorted_df["runtime"])
    plt.xlabel("Config Rank (Best to Worst)")
    plt.ylabel("A* Runtime (sec)")
    plt.title("Runtime Improvement from Tuning")
    plt.grid(True)
    plt.show()

plot_runtime_progression(results_df)

In [ ]:
files.download("hyperparameter_results.csv")
files.download("hyperparameter_results.json")
files.download("final_heuristic_comparison.csv")
files.download("final_heuristic_comparison.json")

In [ ]:
df = pd.read_csv("final_heuristic_comparison.csv")

# Average metrics per heuristic
summary = df.groupby("heuristic").agg({
    "runtime": "mean",
    "path_length": "mean",
    "nodes_expanded": "mean"
}).reset_index()

print("Average Metrics by Heuristic:")
print(summary)

In [ ]:
# Extract dimension from maze filename (assumes naming like 'maze407_dim142.txt')
df["dimension"] = df["maze"].str.extract(r'dim(\d+)').astype(float)

# Average runtime by dimension and heuristic
avg_time_by_dim = df.groupby(["dimension", "heuristic"])["runtime"].mean().reset_index()

# Plot
plt.figure(figsize=(10, 5))
for heuristic in df["heuristic"].unique():
    subset = avg_time_by_dim[avg_time_by_dim["heuristic"] == heuristic]
    plt.plot(subset["dimension"], subset["runtime"], label=heuristic)

plt.xlabel("Maze Dimension")
plt.ylabel("Average A* Runtime (sec)")
plt.title("A* Runtime by Maze Dimension and Heuristic")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
tuning_df = pd.read_csv("hyperparameter_results.csv")

# Sort by runtime for training curve
sorted_tuning = tuning_df.sort_values("runtime").reset_index(drop=True)

plt.figure(figsize=(8, 4))
plt.plot(sorted_tuning.index, sorted_tuning["runtime"])
plt.xlabel("Trial Index (Best to Worst)")
plt.ylabel("A* Runtime with Q-learned Heuristic")
plt.title("Q-learning Heuristic Performance over Tuning Trials")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Fit regressor and extract feature importances
features = ["discount", "explorationProb", "step_size", "num_sample_starts", "step_multiplier"]
X = tuning_df[features]
y = tuning_df["runtime"]

model = RandomForestRegressor(random_state=0)
model.fit(X, y)

importances = model.feature_importances_
importance_df = pd.DataFrame({"parameter": features, "importance": importances}).sort_values("importance", ascending=False)

# Plot heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(importance_df.set_index("parameter").T, annot=True, cmap="Blues", cbar=False)
plt.title("Feature Importance for Runtime Prediction")
plt.tight_layout()
plt.savefig("parameter_importance_heatmap.png")
plt.show()

In [ ]:
# Load and prepare data
df = pd.read_csv("final_heuristic_comparison.csv")
df["dimension"] = df["maze"].str.extract(r'dim(\d+)').astype(float)

# Align runtimes by heuristic per maze
pivot_df = df.pivot(index="maze", columns="heuristic", values=["runtime", "dimension"])
pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]
pivot_df = pivot_df.dropna()
pivot_df["best_traditional"] = pivot_df[["runtime_Manhattan", "runtime_Euclidean"]].min(axis=1)

# Compute relative improvement: (traditional - qlearned) / traditional
pivot_df["relative_improvement"] = (
    (pivot_df["best_traditional"] - pivot_df["runtime_Q-learned"]) / pivot_df["best_traditional"]
)

pivot_df["dimension"] = pivot_df["dimension_Q-learned"]
agg = pivot_df.groupby("dimension")["relative_improvement"].agg(["mean", "count", "std"])
agg["sem"] = agg["std"] / agg["count"]**0.5

plt.figure(figsize=(10, 5))
plt.plot(agg.index, agg["mean"], label="Avg Relative Runtime Improvement", color='green')
plt.fill_between(agg.index, agg["mean"] - agg["sem"], agg["mean"] + agg["sem"], color='green', alpha=0.3)

plt.axhline(0, color='gray', linestyle='--')
plt.xlabel("Maze Dimension")
plt.ylabel("Relative Runtime Improvement (Q - Best of M/E)")
plt.title("Q-learned Heuristic Improvement Over Traditional Heuristics")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("qlearn_vs_traditional_by_dimension.png")
plt.show()

In [ ]:
files.download("hyperparameter_results.csv")
files.download("hyperparameter_results.json")
files.download("final_heuristic_comparison.csv")
files.download("final_heuristic_comparison.json")
files.download("parameter_importance_heatmap.png")
files.download("qlearn_vs_traditional_by_dimension.png")

In [ ]:
# Load your saved results
uploaded = files.upload()
filename = next(iter(uploaded))
results_df = pd.read_csv(filename)

runtimes = results_df["runtime"].values
best_so_far = [runtimes[0]]

for r in runtimes[1:]:
    best_so_far.append(min(best_so_far[-1], r))

# Plot
plt.figure(figsize=(8, 4))
plt.plot(best_so_far)
plt.xlabel("Trial Index (Chronological)")
plt.ylabel("Best Runtime So Far (sec)")
plt.title("Cumulative Best A* Runtime Over Tuning Trials")
plt.grid(True)
plt.tight_layout()
plt.savefig("cumulative_best_runtime.png")
plt.show()


In [ ]:
df["dimension"] = df["maze"].str.extract(r'dim(\d+)').astype(int)

grouped = df.groupby(["heuristic", "dimension"]).agg({
    "runtime": "mean",
    "path_length": "mean",
    "nodes_expanded": "mean"
}).reset_index()

heuristics = grouped["heuristic"].unique()
tables = {}
for h in heuristics:
    table = grouped[grouped["heuristic"] == h].drop(columns="heuristic").sort_values("dimension")
    tables[h] = table
    print(f"\\n📊 Average Metrics per Dimension for {h} Heuristic:")
    display(table)
for h, table in tables.items():
    table.to_csv(f"{h.lower()}_summary_by_dimension.csv", index=False)